# GENERATIVE TEXT TO IMAGE SYNTHESIS PART 2 UPTO EPOCH 25

## IMPORTING LIBRARIES

In [ ]:
import math #FOR MATHEMATICAL FUNCTIONS
import tensorflow.compat.v1 as tf #USIN VERSION 1 OF TENSORFLOW USING COMPATIBILITY OF TENSORFLOW 2
tf.disable_v2_behavior# WE ARE DISABLING THE USE OF TENSORFLOW VERSION 2 AND USING VERSION 1
tf.disable_eager_execution()#DISABLE THE EAGER EXECUTION AS IT WAS CAUSING SOME TROUBLES DURING TRAINING
from tensorflow.python.framework import ops#FOR SOME TENSORFLOW OPERATIONAL FUNCTIONS
import skimage#SCKIT LEARN IMAGE HANDLING LIBRARY
import skimage.io
import imageio#READING IMAGE FILES
import skimage.transform#TRANSFORM IMAGES PROPERTIES
!pip install tf_slim 
import tf_slim as slim# HELPS IN BUILDING COMPLEX MODELS EASILY
import pickle# FOR LOADING THE PICKLES FILES FROM PART 1
import scipy.misc
import random#FOR RANDOM VALUES
import os#FOR OS OPERATIONS
import shutil#FOR HIGH LEVEL FILE OPERATIONS
import numpy as np#NUMERICAL CALCULATIONS LIBBRARY
from os.path import join#TO JOIN FILE PATHS
# to show the loss functions graph over time
!pip install -U tensorboard

## UTILITY FUNCTIONS

In [ ]:

# CLASS FOR BATCH NORMALIZATION 
class batch_norm(object):
    
    def __init__(self, epsilon=1e-5, momentum = 0.9, name="batch_norm"):
        with tf.variable_scope(name):
            self.epsilon = epsilon
            self.momentum = momentum

            self.ema = tf.train.ExponentialMovingAverage(decay=self.momentum)
            self.name = name

#     FUNCTION FOR TRAINING THE GAN MODEL ON THE SKIP THOUGHT VECTORS
    def __call__(self, x, train=True):
        shape = x.get_shape().as_list()

        if train:
            with tf.variable_scope(self.name) as scope:
                self.beta = tf.get_variable("beta", [shape[-1]],
                                    initializer=tf.constant_initializer(0.))
                self.gamma = tf.get_variable("gamma", [shape[-1]],
                                    initializer=tf.random_normal_initializer(1., 0.02))

                try:
                    batch_mean, batch_var = tf.nn.moments(x, [0, 1, 2], name='moments')
                except:
                    batch_mean, batch_var = tf.nn.moments(x, [0, 1], name='moments')

                ema_apply_op = self.ema.apply([batch_mean, batch_var])
                self.ema_mean, self.ema_var = self.ema.average(batch_mean), self.ema.average(batch_var)

                with tf.control_dependencies([ema_apply_op]):
                    mean, var = tf.identity(batch_mean), tf.identity(batch_var)
        else:
            mean, var = self.ema_mean, self.ema_var

        normed = tf.nn.batch_norm_with_global_normalization(
                x, mean, var, self.beta, self.gamma, self.epsilon, scale_after_normalization=True)

        return normed


# FUNCTION TO COMPUTE BINARY CROSS ENTROPY
def binary_cross_entropy(preds, targets, name=None):
    eps = 1e-12
    with ops.op_scope([preds, targets], name, "bce_loss") as name:
        preds = ops.convert_to_tensor(preds, name="preds")
        targets = ops.convert_to_tensor(targets, name="targets")
        return tf.reduce_mean(-(targets * tf.log(preds + eps) +
                              (1. - targets) * tf.log(1. - preds + eps)))
# Concatenate conditioning vector on feature map axis
def conv_cond_concat(x, y):
    x_shapes = x.get_shape()
    y_shapes = y.get_shape()
    return tf.concat(3, [x, y*tf.ones([x_shapes[0], x_shapes[1], x_shapes[2], y_shapes[3]])])

# FUNCTION FOR CONVOLUTION2D LAYER
def conv2d(input_, output_dim, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
           name="conv2d"):
    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim], initializer = tf.keras.initializers.glorot_normal())
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='SAME')

        biases = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))
        conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())

        return conv

# FUNCTION FOR DECONVOLUTION2D LAYER
def deconv2d(input_, output_shape, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
             name="deconv2d", with_w=False):
    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_h, output_shape[-1], input_.get_shape()[-1]], initializer = tf.keras.initializers.glorot_normal())
        try:
            deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape,
                                strides=[1, d_h, d_w, 1])

        except AttributeError:
            deconv = tf.nn.deconv2d(input_, w, output_shape=output_shape,
                                strides=[1, d_h, d_w, 1])

        biases = tf.get_variable('biases', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

        if with_w:
            return deconv, w, biases
        else:
            return deconv

# FUNCTION FOR LEAKY RELU FUNCTION
def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

# FUNCTION FOR LINEAR LAYER
def linear(input_, output_size, scope=None, stddev=0.02, bias_start=0.0,
           with_w=False):
    shape = input_.get_shape().as_list()

    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32,
                                 tf.keras.initializers.glorot_normal())
        bias = tf.get_variable("bias", [output_size],
            initializer=tf.constant_initializer(bias_start))
        if with_w:
            return tf.matmul(input_, matrix) + bias, matrix, bias
        else:
            return tf.matmul(input_, matrix) + bias



# FUNCTION FORCALLING THE GENERATOR
def get_gt(batch_size, classes, real=1, name="gt"):

    with tf.variable_scope(name, reuse=None):
        r_f = tf.get_variable("rf", [batch_size, 1],
                               initializer = tf.constant_initializer(
                                   real))
        gt = tf.concat(1, [r_f, classes], name = 'gt_concat_classes')
        return gt

## FUNCTIONS FOR IMAGE PROCESSING

In [ ]:

#FUCNTION TO LOAD ALL THE TRAIN IMAGE FILES
def load_image_array_flowers(image_file, image_size):
    img = skimage.io.imread(image_file)
    # GRAYSCALE THE IMAGES
    if len(img.shape) == 2:
        img_new = np.ndarray( (img.shape[0], img.shape[1], 3), dtype = 'uint8')
        img_new[:,:,0] = img
        img_new[:,:,1] = img
        img_new[:,:,2] = img
        img = img_new

    img_resized = skimage.transform.resize(img, (image_size, image_size))

    # FLIP HORIZONTAL WIRH A PROBABILITY  OF 0.5
    if random.random() > 0.5:
        img_resized = np.fliplr(img_resized)


    return img_resized.astype('float32')

#FUCNTION TO LOAD ALL THE VALIDATION/TRAIN IMAGE FILES
def load_image_array(image_file, image_size,
                     image_id, data_dir='Data/datasets/mscoco/train2014',
                     mode='train'):
    img = None
    if os.path.exists(image_file):
        #print('found' + image_file)
        img = skimage.io.imread(image_file)
    else:
        print('notfound' + image_file)
        img = skimage.io.imread('http://mscoco.org/images/%d' % (image_id))
        img_path = os.path.join(data_dir, 'COCO_%s2014_%.12d.jpg' % ( mode,
                                                                      image_id))
        imageio.imwrite(img_path, img)

    # GRAYSCALE
    if len(img.shape) == 2:
        img_new = np.ndarray( (img.shape[0], img.shape[1], 3), dtype = 'uint8')
        img_new[:,:,0] = img
        img_new[:,:,1] = img
        img_new[:,:,2] = img
        img = img_new

    img_resized = skimage.transform.resize(img, (image_size, image_size))

    # FLIP HORIZONTAL WIRH A PROBABILITY 0.5
    if random.random() > 0.5:
        img_resized = np.fliplr(img_resized)

    return img_resized.astype('float32')


## FUNCTION TO CREATE GAN MODEL

In [ ]:

# CLASS FOR CREATING THE GAN
class GAN :

    
#     FUNCTION FOR GAN MODEL INITILIZATION
    def __init__(self, options) :
        self.options = options

#     FUNCTION TO BUILD THE MODEL
    def build_model(self) :

        
        print('Initializing placeholder')
        img_size = self.options['image_size']
        t_real_image = tf.placeholder('float32', [self.options['batch_size'],
                                      img_size, img_size, 3],
                                      name = 'real_image')
        t_wrong_image = tf.placeholder('float32', [self.options['batch_size'],
                                       img_size, img_size, 3],
                                       name = 'wrong_image')

        t_real_caption = tf.placeholder('float32', [self.options['batch_size'],
                                        self.options['caption_vector_length']],
                                        name='real_captions')

        t_z = tf.placeholder('float32', [self.options['batch_size'],
                              self.options['z_dim']], name='input_noise')

        t_real_classes = tf.placeholder('float32', [self.options['batch_size'],
                                        self.options['n_classes']],
                                        name='real_classes')

        t_wrong_classes = tf.placeholder('float32', [self.options['batch_size'],
                                         self.options['n_classes']],
                                         name='wrong_classes')

        t_training = tf.placeholder(tf.bool, name='training')

#         BUILDING THE GENERATOR
        print('Building the Generator')
        fake_image = self.generator(t_z, t_real_caption,
                                                 t_training)

#         BUILDING THE DISCRIMINATOR
        print('Building the Discriminator')
        disc_real_image, disc_real_image_logits, disc_real_image_aux, \
            disc_real_image_aux_logits = self.discriminator(
                t_real_image, t_real_caption, self.options['n_classes'],
                t_training)

        disc_wrong_image, disc_wrong_image_logits, disc_wrong_image_aux, \
            disc_wrong_image_aux_logits  = self.discriminator(
                t_wrong_image, t_real_caption, self.options['n_classes'],
                t_training, reuse = True)

        disc_fake_image, disc_fake_image_logits, disc_fake_image_aux, \
            disc_fake_image_aux_logits  = self.discriminator(
                fake_image, t_real_caption, self.options['n_classes'],
                t_training, reuse = True)

        d_right_predictions = tf.equal(tf.argmax(disc_real_image_aux, 1),
                                       tf.argmax(t_real_classes, 1))
        d_right_accuracy = tf.reduce_mean(tf.cast(d_right_predictions,
                                                  tf.float32))

        d_wrong_predictions = tf.equal(tf.argmax(disc_wrong_image_aux, 1),
                                       tf.argmax(t_wrong_classes, 1))
        d_wrong_accuracy = tf.reduce_mean(tf.cast(d_wrong_predictions,
                                                  tf.float32))

        d_fake_predictions = tf.equal(tf.argmax(disc_fake_image_aux_logits, 1),
                                      tf.argmax(t_real_classes, 1))
        d_fake_accuracy = tf.reduce_mean(tf.cast(d_fake_predictions,
                                                 tf.float32))

        tf.get_variable_scope()._reuse = False

#         BUILDING THE LOSS FUNCTION FOR THE GENERATOR AND DISCRIMINATOR
        print('Building the Loss Function')
        g_loss_1 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                                  logits=disc_fake_image_logits,
                                  labels=tf.ones_like(disc_fake_image)))

        g_loss_2 = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                                            logits=disc_fake_image_aux_logits,
                                            labels=t_real_classes))

        d_loss1 = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                                        logits=disc_real_image_logits,
                                        labels=tf.ones_like(disc_real_image)))
        d_loss1_1 = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                                            logits=disc_real_image_aux_logits,
                                            labels=t_real_classes))
        d_loss2 = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                                    logits=disc_wrong_image_logits,
                                    labels=tf.zeros_like(disc_wrong_image)))
        d_loss2_1 = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                                            logits=disc_wrong_image_aux_logits,
                                            labels=t_wrong_classes))
        d_loss3 = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                                        logits=disc_fake_image_logits,
                                        labels=tf.zeros_like(disc_fake_image)))

        d_loss = d_loss1 + d_loss1_1 + d_loss2 + d_loss2_1 + d_loss3 + g_loss_2

        g_loss = g_loss_1 + g_loss_2

        t_vars = tf.trainable_variables()
        
#         PRINT ALL VALUES: GENERATOR/DISCRIMINATOR LOSS, GENERATOR/DISCRIMINATOR LOSS ON REAL/FAKE IMAGE .ETC
        print('List of all variables')
        for v in t_vars:
            print(v.name)
            print(v)
            self.add_histogram_summary(v.name, v)

        self.add_tb_scalar_summaries(d_loss, g_loss, d_loss1, d_loss2, d_loss3,
              d_loss1_1, d_loss2_1, g_loss_1, g_loss_2, d_right_accuracy,
              d_wrong_accuracy, d_fake_accuracy)

        self.add_image_summary('Generated Images', fake_image,
                               self.options['batch_size'])

        d_vars = [var for var in t_vars if 'd_' in var.name]
        g_vars = [var for var in t_vars if 'g_' in var.name]

        input_tensors = {
            't_real_image' : t_real_image,
            't_wrong_image' : t_wrong_image,
            't_real_caption' : t_real_caption,
            't_z' : t_z,
            't_real_classes' : t_real_classes,
            't_wrong_classes' : t_wrong_classes,
            't_training' : t_training,

        }

        variables = {
            'd_vars' : d_vars,
            'g_vars' : g_vars
        }

        loss = {
            'g_loss' : g_loss,
            'd_loss' : d_loss
        }

        outputs = {
            'generator' : fake_image
        }

        checks = {
            'd_loss1': d_loss1,
            'd_loss2': d_loss2,
            'd_loss3': d_loss3,
            'g_loss_1': g_loss_1,
            'g_loss_2': g_loss_2,
            'd_loss1_1': d_loss1_1,
            'd_loss2_1': d_loss2_1,
            'disc_real_image_logits': disc_real_image_logits,
            'disc_wrong_image_logits': disc_wrong_image,
            'disc_fake_image_logits': disc_fake_image_logits
        }

        return input_tensors, variables, loss, outputs, checks

    
    #ADDING THE METRICS DURING TRAINING FOR EACH EPOCH
    def add_tb_scalar_summaries(self, d_loss, g_loss, d_loss1, d_loss2,
                                  d_loss3, d_loss1_1, d_loss2_1, g_loss_1,
                                  g_loss_2, d_right_accuracy,
                                  d_wrong_accuracy, d_fake_accuracy):

        self.add_scalar_summary("D_Loss", d_loss)
        self.add_scalar_summary("G_Loss", g_loss)
        self.add_scalar_summary("D loss-1 [Real/Fake loss for real images]",
                                d_loss1)
        self.add_scalar_summary("D loss-2 [Real/Fake loss for wrong images]",
                                d_loss2)
        self.add_scalar_summary("D loss-3 [Real/Fake loss for fake images]",
                                d_loss3)
        self.add_scalar_summary(
            "D loss-4 [Aux Classifier loss for real images]", d_loss1_1)
        self.add_scalar_summary(
            "D loss-5 [Aux Classifier loss for wrong images]", d_loss2_1)
        self.add_scalar_summary("G loss-1 [Real/Fake loss for fake images]",
                                g_loss_1)
        self.add_scalar_summary(
            "G loss-2 [Aux Classifier loss for fake images]", g_loss_2)
        self.add_scalar_summary("Discriminator Real Image Accuracy",
                                d_right_accuracy)
        self.add_scalar_summary("Discriminator Wrong Image Accuracy",
                                d_wrong_accuracy)
        self.add_scalar_summary("Discriminator Fake Image Accuracy",
                                d_fake_accuracy)

#     FUNCTION TO PRINT IN SCALAR FORMAT
    def add_scalar_summary(self, name, var):
        with tf.name_scope('summaries'):
            tf.summary.scalar(name, var)

#     FUNCTION TO PRINT IN HISTGRAM FORMAT
    def add_histogram_summary(self, name, var):
        with tf.name_scope('summaries'):
            tf.summary.histogram(name, var)

#     FUNCTION TO PRINT IN IMAGE FORMAT
    def add_image_summary(self, name, var, max_outputs=1):
        with tf.name_scope('summaries'):
            tf.summary.image(name, var, max_outputs=max_outputs)

    # GENERATOR IMPLEMENTATION FUNCTION
    def generator(self, t_z, t_text_embedding, t_training):

        s = self.options['image_size']
        s2, s4, s8, s16 = int(s / 2), int(s / 4), int(s / 8), int(s / 16)

        reduced_text_embedding = lrelu(
            linear(t_text_embedding, self.options['t_dim'], 'g_embedding'))
        z_concat = tf.concat([t_z, reduced_text_embedding], -1)
        z_ = linear(z_concat, self.options['gf_dim'] * 8 * s16 * s16,
                        'g_h0_lin')
        h0 = tf.reshape(z_, [-1, s16, s16, self.options['gf_dim'] * 8])
        h0 = tf.nn.relu(slim.batch_norm(h0, is_training = t_training,
                                        scope="g_bn0"))

        h1 = deconv2d(h0, [self.options['batch_size'], s8, s8,
                               self.options['gf_dim'] * 4], name = 'g_h1')
        h1 = tf.nn.relu(slim.batch_norm(h1, is_training = t_training,
                                        scope="g_bn1"))

        h2 = deconv2d(h1, [self.options['batch_size'], s4, s4,
                               self.options['gf_dim'] * 2], name = 'g_h2')
        h2 = tf.nn.relu(slim.batch_norm(h2, is_training = t_training,
                                        scope="g_bn2"))

        h3 = deconv2d(h2, [self.options['batch_size'], s2, s2,
                               self.options['gf_dim'] * 1], name = 'g_h3')
        h3 = tf.nn.relu(slim.batch_norm(h3, is_training = t_training,
                                        scope="g_bn3"))

        h4 = deconv2d(h3, [self.options['batch_size'], s, s, 3],
                          name = 'g_h4')
        return (tf.tanh(h4) / 2. + 0.5)


    # DISCRIMINATOR IMPLEMENTATION FUNCTION
    
    def discriminator(self, image, t_text_embedding, n_classes, t_training,
                      reuse = False) :
        if reuse :
            tf.get_variable_scope().reuse_variables()

        h0 = lrelu(
            conv2d(image, self.options['df_dim'], name = 'd_h0_conv'))  # 64

        h1 = lrelu(slim.batch_norm(conv2d(h0,
                                             self.options['df_dim'] * 2,
                                             name = 'd_h1_conv'),
                                       reuse=reuse,
                                       is_training = t_training,
                                       scope = 'd_bn1'))  # 32

        h2 = lrelu(slim.batch_norm(conv2d(h1,
                                             self.options['df_dim'] * 4,
                                             name = 'd_h2_conv'),
                                       reuse=reuse,
                                       is_training = t_training,
                                       scope = 'd_bn2'))  # 16
        h3 = lrelu(slim.batch_norm(conv2d(h2,
                                             self.options['df_dim'] * 8,
                                             name = 'd_h3_conv'),
                                       reuse=reuse,
                                       is_training = t_training,
                                       scope = 'd_bn3'))  # 8
        h3_shape = h3.get_shape().as_list()
        # ADD TEXT EMBEDDING TO THE NETWORK
        reduced_text_embeddings = lrelu(linear(t_text_embedding,
                                                       self.options['t_dim'],
                                                       'd_embedding'))
        reduced_text_embeddings = tf.expand_dims(reduced_text_embeddings, 1)
        reduced_text_embeddings = tf.expand_dims(reduced_text_embeddings, 2)
        tiled_embeddings = tf.tile(reduced_text_embeddings,
                                   [1, h3_shape[1], h3_shape[1], 1],
                                   name = 'tiled_embeddings')

        h3_concat = tf.concat([h3, tiled_embeddings], 3, name = 'h3_concat')
        h3_new = lrelu(slim.batch_norm(conv2d(h3_concat,
                                                self.options['df_dim'] * 8,
                                                      1, 1, 1, 1,
                                                name = 'd_h3_conv_new'),
                                        reuse=reuse,
                                        is_training = t_training,
                                        scope = 'd_bn4'))  # 4

        h3_flat = tf.reshape(h3_new, [self.options['batch_size'], -1])

        h4 = linear(h3_flat, 1, 'd_h4_lin_rw')
        h4_aux = linear(h3_flat, n_classes, 'd_h4_lin_ac')

        return tf.nn.sigmoid(h4), h4, tf.nn.sigmoid(h4_aux), h4_aux



## FUNCTION TO PERFORM TRAINING OF GAN MODEL

In [ ]:
# INITIALIZING THE PARAMETERS

#     PARAMEETERS USED
#     z_dim : Noise dimension
#     t_dim : Text feature dimension
#     image_size : Image Dimension
#     gf_dim : Number of conv in the first layer generator
#     df_dim : Number of conv in the first layer discriminator
#     gfc_dim : Dimension of gen untis for for fully connected layer
#     caption_vector_length : Caption Vector Length
#     batch_size : Batch Size


z_dim=100

t_dim=256

batch_size=64

image_size=128

gf_dim=64

df_dim=64

caption_vector_length=4800

n_classes = 102 #THAT IS IN FLOWER'S DATASET WE DOWNLOAD THERE WERE 102 CLASSES OF FLOWERS

#LOCATION OF THE DATA
data_dir="/kaggle/input/generative-text-to-image-synthesis-part-1/Data/Flowers" 

#LOCATION OF THE DATA TO STORE IMAGE GENERATED
data_dir_output='Data/Flowers'

learning_rate=0.0002 #LEARNING RATE OF MODEL

beta1=0.5

epochs=25 #NUMBER OF EPOCHS

save_every=30 #SAVE IMAGES GENERATED BY MODEL FROM SENTECES AFETR EVERY 30 BATCHES IN EACH EPOCH 
                     

resume_model=False #WANT TO RESUME MODEL TRAINING AFTER A CERYAIN EPOCH

data_set= "flowers" #DATASET NAME

model_name="GENERATIVE TEXT TO IMAGE SYNTHESIS GAN MODEL" #NAME TO GIVE MODEL

train = True

# MAIN FUNCTIOON FOR THE MODEL TRAINING
def main():
    
    
#     LOADING MODEL PARAMETERS AND DIRECTORIES
    model_dir, model_chkpnts_dir, model_samples_dir, model_val_samples_dir,\
                            model_summaries_dir = initialize_directories()

    datasets_root_dir = join(data_dir, '')
    loaded_data = load_training_data(datasets_root_dir, data_set,
                                     caption_vector_length,
                                     n_classes)
    model_options = {
        'z_dim': z_dim,
        't_dim': t_dim,
        'batch_size': batch_size,
        'image_size': image_size,
        'gf_dim': gf_dim,
        'df_dim': df_dim,
        'caption_vector_length': caption_vector_length,
        'n_classes': loaded_data['n_classes']
    }

    # Initialize and build the GAN model
    gan = GAN(model_options)
    input_tensors, variables, loss, outputs, checks = gan.build_model()

#     MODEL OPTIMIZER'S FOR DISCRIMINATORE
    d_optim = tf.train.AdamOptimizer(learning_rate,
                                     beta1=beta1).minimize(loss['d_loss'],
                                            var_list=variables['d_vars'])
#     MODEL OPTIMIZER'S FOR GENERATOR
    g_optim = tf.train.AdamOptimizer(learning_rate,
                                     beta1=beta1).minimize(loss['g_loss'],
                                            var_list=variables['g_vars'])

    global_step_tensor = tf.Variable(1, trainable=False, name='global_step')
    merged = tf.summary.merge_all()
    sess = tf.InteractiveSession()

#     OBJECT FOR SAVING THE MODEL TRAIN HISTORY AS SUMMARY
    summary_writer = tf.summary.FileWriter(model_summaries_dir, sess.graph)

    tf.global_variables_initializer().run()
    saver = tf.train.Saver(max_to_keep=10000)

#     CHECK WHETHER TO START TRAINING FROM BEGINNING OR RESUME FROM LAST EPOCH'S
    if resume_model:
        print('Trying to resume training from a previous checkpoint' +
              str(tf.train.latest_checkpoint(model_chkpnts_dir)))
        if tf.train.latest_checkpoint(model_chkpnts_dir) is not None:
            saver.restore(sess, tf.train.latest_checkpoint(model_chkpnts_dir))
            print('Successfully loaded model. Resuming training.')
        else:
            print('Could not load checkpoints.  Training a new model')
    global_step = global_step_tensor.eval()
    gs_assign_op = global_step_tensor.assign(global_step)
    for i in range(epochs):
        batch_no = 0
        while batch_no * batch_size + batch_size < \
                loaded_data['data_length']:

            real_images, wrong_images, caption_vectors, z_noise, image_files, \
            real_classes, wrong_classes, image_caps, image_ids = \
                               get_training_batch(batch_no, batch_size,
                                                  image_size, z_dim,
                                                  'train', datasets_root_dir,
                                                  data_set, loaded_data)

            # DISCR UPDATE
            check_ts = [checks['d_loss1'], checks['d_loss2'],
                        checks['d_loss3'], checks['d_loss1_1'],
                        checks['d_loss2_1']]

            feed = {
                input_tensors['t_real_image'].name : real_images,
                input_tensors['t_wrong_image'].name : wrong_images,
                input_tensors['t_real_caption'].name : caption_vectors,
                input_tensors['t_z'].name : z_noise,
                input_tensors['t_real_classes'].name : real_classes,
                input_tensors['t_wrong_classes'].name : wrong_classes,
                input_tensors['t_training'].name : train
            }

            _, d_loss, gen, d1, d2, d3, d4, d5= sess.run([d_optim,
                        loss['d_loss'],outputs['generator']] + check_ts,
                        feed_dict=feed)

#             printing the status of model training
            print("D total loss: {}\n"
                  "D loss-1 [Real/Fake loss for real images] : {} \n"
                  "D loss-2 [Real/Fake loss for wrong images]: {} \n"
                  "D loss-3 [Real/Fake loss for fake images]: {} \n"
                  "D loss-4 [Aux Classifier loss for real images]: {} \n"
                  "D loss-5 [Aux Classifier loss for wrong images]: {}"
                  " ".format(d_loss, d1, d2, d3, d4, d5))

            # UPDATING THE GENRATOR LOSSE'S
            _, g_loss, gen = sess.run([g_optim, loss['g_loss'],
                                       outputs['generator']], feed_dict=feed)
            

            # UPDATING THE GENRATOR LOSSE'S SECOND TIME
            
            _, summary, g_loss, gen, g1, g2 = sess.run([g_optim, merged,
                   loss['g_loss'], outputs['generator'], checks['g_loss_1'],
                   checks['g_loss_2']], feed_dict=feed)
            
            
#             SAVING THE METRICS OF GENERATOR/DISCRIMINATOR AND THERE LOSSE'S
            summary_writer.add_summary(summary, global_step)
            print("\n\nLOSSES\nDiscriminator Loss: {}\nGenerator Loss: {"
                  "}\nBatch Number: {}\nEpoch: {},\nTotal Batches per "
                  "epoch: {}\n".format( d_loss, g_loss, batch_no, i,
                    int(len(loaded_data['image_list']) / batch_size)))
            print("\nG loss-1 [Real/Fake loss for fake images] : {} \n"
                  "G loss-2 [Aux Classifier loss for fake images]: {} \n"
                  " ".format(g1, g2))
            global_step += 1
            sess.run(gs_assign_op)
            batch_no += 1
#             SAVING THE IMAGES CREATED BY MODEL FROM THE CAPTIONS AND THE CHECKPOINT FOR MODEL TRAINING
            if (batch_no % save_every) == 0 and batch_no != 0:
                print("Saving Images and the Model\n\n")

                save_for_vis(model_samples_dir, real_images, gen, image_files,
                             image_caps, image_ids)
                save_path = saver.save(sess,
                                       join(model_chkpnts_dir,
                                            "latest_model_{}_temp.ckpt".format(
                                                data_set)))

                # Getting a batch for validation of the model ggenerated iamegs
                val_captions, val_image_files, val_image_caps, val_image_ids = \
                          get_val_caps_batch(batch_size, loaded_data,
                                             data_set, datasets_root_dir)

                shutil.rmtree(model_val_samples_dir)
                os.makedirs(model_val_samples_dir)

                for val_viz_cnt in range(0, 4):
                    val_z_noise = np.random.uniform(-1, 1, [batch_size,
                                                            z_dim])

                    val_feed = {
                        input_tensors['t_real_caption'].name : val_captions,
                        input_tensors['t_z'].name : val_z_noise,
                        input_tensors['t_training'].name : True
                    }

                    val_gen = sess.run([outputs['generator']],
                                       feed_dict=val_feed)
                    save_for_viz_val(model_val_samples_dir, val_gen,
                                     val_image_files, val_image_caps,
                                     val_image_ids, image_size,
                                     val_viz_cnt)

        # Save the model after every epoch
        if i % 1 == 0:
            epoch_dir = join(model_chkpnts_dir, str(i))
            if not os.path.exists(epoch_dir):
                os.makedirs(epoch_dir)

            save_path = saver.save(sess,
                                   join(epoch_dir,
                                        "model_after_{}_epoch_{}.ckpt".
                                            format(data_set, i)))
            val_captions, val_image_files, val_image_caps, val_image_ids = \
                  get_val_caps_batch(batch_size, loaded_data,
                                     data_set, datasets_root_dir)

            shutil.rmtree(model_val_samples_dir)
            os.makedirs(model_val_samples_dir)

            for val_viz_cnt in range(0, 10):
                val_z_noise = np.random.uniform(-1, 1, [batch_size,
                                                        z_dim])
                val_feed = {
                    input_tensors['t_real_caption'].name : val_captions,
                    input_tensors['t_z'].name : val_z_noise,
                    input_tensors['t_training'].name : True
                }
                val_gen = sess.run([outputs['generator']], feed_dict=val_feed)
                save_for_viz_val(model_val_samples_dir, val_gen,
                                 val_image_files, val_image_caps,
                                 val_image_ids, image_size,
                                 val_viz_cnt)

# FUNCTION TO LOAD TRAIN DATA
def load_training_data(data_dir, data_set, caption_vector_length, n_classes) :
    if data_set == 'flowers' :
        flower_str_captions = pickle.load(
            open(join(data_dir, 'pickles/flowers_caps.pkl'), "rb"))

        img_classes = pickle.load(
            open(join(data_dir, 'pickles/flower_tc.pkl'), "rb"))

        flower_enc_captions = pickle.load(
            open(join(data_dir, 'pickles/flower_tv.pkl'), "rb"))
        tr_image_ids = pickle.load(
            open(join(data_dir, 'pickles/train_ids.pkl'), "rb"))
        val_image_ids = pickle.load(
            open(join(data_dir, 'pickles/val_ids.pkl'), "rb"))

        max_caps_len = caption_vector_length
        tr_n_imgs = len(tr_image_ids)
        val_n_imgs = len(val_image_ids)

        return {
            'image_list'    : tr_image_ids,
            'captions'      : flower_enc_captions,
            'data_length'   : tr_n_imgs,
            'classes'       : img_classes,
            'n_classes'     : n_classes,
            'max_caps_len'  : max_caps_len,
            'val_img_list'  : val_image_ids,
            'val_captions'  : flower_enc_captions,
            'val_data_len'  : val_n_imgs,
            'str_captions'  : flower_str_captions
        }

    else :
        raise Exception('No Dataset Found')

# FUNCTION TO CRAETE THE DIRECTORIES FOR STORING THE CHECKPOINT'S, GENERATED IMAGES, SUMMARIES
def initialize_directories():
    model_dir = join(data_dir_output, 'training', model_name)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    model_chkpnts_dir = join(model_dir, 'checkpoints')
    if not os.path.exists(model_chkpnts_dir):
        os.makedirs(model_chkpnts_dir)

    model_summaries_dir = join(model_dir, 'summaries')
    if not os.path.exists(model_summaries_dir):
        os.makedirs(model_summaries_dir)

    model_samples_dir = join(model_dir, 'samples')
    if not os.path.exists(model_samples_dir):
        os.makedirs(model_samples_dir)

    model_val_samples_dir = join(model_dir, 'val_samples')
    if not os.path.exists(model_val_samples_dir):
        os.makedirs(model_val_samples_dir)

    return model_dir, model_chkpnts_dir, model_samples_dir, \
           model_val_samples_dir, model_summaries_dir

# FUNCTIONS FOR SAVING THE IMAGES GENERATED BY THE IMAGE CAPTIONS AND THE GENERATOR ALSO
def save_for_viz_val(data_dir, generated_images, image_files, image_caps,
                     image_ids, image_size, id):

    generated_images = np.squeeze(np.array(generated_images))
    for i in range(0, generated_images.shape[0]) :
        image_dir = join(data_dir, str(image_ids[i]))
        if not os.path.exists(image_dir):
            os.makedirs(image_dir)

        real_image_path = join(image_dir,
                               '{}.jpg'.format(image_ids[i]))
        if os.path.exists(image_dir):
            real_images_255 = load_image_array(image_files[i],
                                        image_size, image_ids[i], mode='val')
            imageio.imwrite(real_image_path, real_images_255)

        caps_dir = join(image_dir, "caps.txt")
        if not os.path.exists(caps_dir):
            with open(caps_dir, "w") as text_file:
                text_file.write(image_caps[i]+"\n")

        fake_images_255 = generated_images[i]
        imageio.imwrite(join(image_dir, 'fake_image_{}.jpg'.format(id)),
                          fake_images_255)

# FUNCTIONS FOR SAVING THE IMAGES GENERATED BY THE IMAGE CAPTIONS AND THE GENERATOR ALSO
def save_for_vis(data_dir, real_images, generated_images, image_files,
                 image_caps, image_ids) :

    shutil.rmtree(data_dir)
    os.makedirs(data_dir)

    for i in range(0, real_images.shape[0]) :
        real_images_255 = (real_images[i, :, :, :])
        imageio.imwrite(join(data_dir,
               '{}_{}.jpg'.format(i, image_files[i].split('/')[-1])),
                          real_images_255)

        fake_images_255 = (generated_images[i, :, :, :])
        imageio.imwrite(join(data_dir, 'fake_image_{}.jpg'.format(
            i)), fake_images_255)

    str_caps = '\n'.join(image_caps)
    str_image_ids = '\n'.join([str(image_id) for image_id in image_ids])
    with open(join(data_dir, "caps.txt"), "w") as text_file:
        text_file.write(str_caps)
    with open(join(data_dir, "ids.txt"), "w") as text_file:
        text_file.write(str_image_ids)

# FUNCTION TO GET CAPTION'S BATCH FOR VALIDATION
def get_val_caps_batch(batch_size, loaded_data, data_set, data_dir):
    if data_set == 'flowers':
        captions = np.zeros((batch_size, loaded_data['max_caps_len']))

        batch_idx = np.random.randint(0, loaded_data['val_data_len'],
                                      size = batch_size)
        image_ids = np.take(loaded_data['val_img_list'], batch_idx)
        image_files = []
        image_caps = []
        for idx, image_id in enumerate(image_ids) :
            image_file = join(data_dir,
                              '102flowers/jpg/' + image_id)
            random_caption = random.randint(0, 4)
            captions[idx, :] = \
                loaded_data['val_captions'][image_id][random_caption][
                0 :loaded_data['max_caps_len']]

            image_caps.append(loaded_data['str_captions']
                              [image_id][random_caption])
            image_files.append(image_file)

        return captions, image_files, image_caps, image_ids
    else:
        raise Exception('Dataset not found')

# FUNCTION TO GET CAPTION'S BATCH FOR TRAINING
def get_training_batch(batch_no, batch_size, image_size, z_dim, split,
                       data_dir, data_set, loaded_data = None) :
    
    if data_set == 'flowers':
        real_images = np.zeros((batch_size, image_size, image_size, 3))
        wrong_images = np.zeros((batch_size, image_size, image_size, 3))
        captions = np.zeros((batch_size, loaded_data['max_caps_len']))
        real_classes = np.zeros((batch_size, loaded_data['n_classes']))
        wrong_classes = np.zeros((batch_size, loaded_data['n_classes']))

        cnt = 0
        image_files = []
        image_caps = []
        image_ids = []
        for i in range(batch_no * batch_size,
                       batch_no * batch_size + batch_size) :
            idx = i % len(loaded_data['image_list'])
            image_file = join(data_dir,
                              '102flowers/jpg/' + loaded_data['image_list'][idx])

            image_ids.append(loaded_data['image_list'][idx])

            image_array = load_image_array_flowers(image_file,
                                                            image_size)
            real_images[cnt, :, :, :] = image_array

            # Improve this selection of wrong image
            wrong_image_id = random.randint(0,
                                            len(loaded_data['image_list']) - 1)
            wrong_image_file = join(data_dir,
                                    '102flowers/jpg/' + loaded_data['image_list'][
                                                            wrong_image_id])
            wrong_image_array = load_image_array_flowers(wrong_image_file,
                                                                  image_size)
            wrong_images[cnt, :, :, :] = wrong_image_array

            wrong_classes[cnt, :] = loaded_data['classes'][loaded_data['image_list'][
                                    wrong_image_id]][0 :loaded_data['n_classes']]

            random_caption = random.randint(0, 4)
            captions[cnt, :] = \
            loaded_data['captions'][loaded_data['image_list'][idx]][
                                random_caption][0 :loaded_data['max_caps_len']]

            real_classes[cnt, :] = \
                loaded_data['classes'][loaded_data['image_list'][idx]][
                                                0 :loaded_data['n_classes']]
            str_cap = loaded_data['str_captions'][loaded_data['image_list']
                                [idx]][random_caption]

            image_files.append(image_file)
            image_caps.append(str_cap)
            cnt += 1

        z_noise = np.random.uniform(-1, 1, [batch_size, z_dim])
        return real_images, wrong_images, captions, z_noise, image_files, \
               real_classes, wrong_classes, image_caps, image_ids
    else:
        raise Exception('Dataset not found')




## STARTING THE TRAINING OF THE GAN MODEL

In [ ]:
# FUNCTION CALLING THE MAIN TRAINING FUNCTION
main()

## SHOWING THE RESULTS IN TENSORBOARD

In [3]:
#UPLOADING THE SUMMARY OF THE TRAINING TILL 25 EPOCH'S TO TENSORBOARD

!timeout 30m bash -c "echo yes | tensorboard dev upload\
--logdir 'Data/Flowers/training/GENERATIVE TEXT TO IMAGE SYNTHESIS GAN MODEL/summaries'\
--name 'GENERATIVE TEXT TO IMAGE SYNTHESIS PART 2 EPOCH 25 RESULTS.'\
--description 'THIS IS THE SUMMARY FOR THE RESULTS UPTO 25th EPOCH.'"


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

Data/Flowers/training/GENERATIVE TEXT TO IMAGE SYNTHESIS GAN MODEL/summaries

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) 
To sign in with the TensorBoard uploader:

1. On your computer or phone, visit:

   https://www.google.com/device

2. Sign in with your Google account, then enter:

   YBJM-BVPJ

^C
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File 

# 

# ------------------------------------------------THE END------------------------------------------------